In [1]:
!pip install --upgrade scikit-learn imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.13.0
    Uninstalling imbalanced-learn-0.13.0:
      Successfully uninstalled imbalanced-learn-0.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.7.2 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.2 which

In [2]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
import time
import os

print("✓ All libraries imported successfully!")


✓ All libraries imported successfully!


In [3]:
# Data loading 
train = pd.read_csv("/kaggle/input/playground-series-s5e11/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s5e11/test.csv")
print("Data Is Loaded Successfully...")

Data Is Loaded Successfully...


In [4]:
#Displaying the Data 
print("Displaying The Data...")
print(f"\nThe Train Shape:{train.shape}")
print(f"The Test Shape:{test.shape}")

print("\n---First Few Rows of Data---")
print(train.head())

print("\n---Last Few Rows of Data---")
print(train.tail())
      
print("\n--- Column data types ---")
print(train.dtypes)

print("\n--- Missing values in train ---")
print(train.isnull().sum())
print("\n--- Missing values in test ---")
print(test.isnull().sum())
print("Data Is Displayed")


Displaying The Data...

The Train Shape:(593994, 13)
The Test Shape:(254569, 12)

---First Few Rows of Data---
   id  annual_income  debt_to_income_ratio  credit_score  loan_amount  \
0   0       29367.99                 0.084           736      2528.42   
1   1       22108.02                 0.166           636      4593.10   
2   2       49566.20                 0.097           694     17005.15   
3   3       46858.25                 0.065           533      4682.48   
4   4       25496.70                 0.053           665     12184.43   

   interest_rate  gender marital_status education_level employment_status  \
0          13.67  Female         Single     High School     Self-employed   
1          12.92    Male        Married        Master's          Employed   
2           9.76    Male         Single     High School          Employed   
3          16.10  Female         Single     High School          Employed   
4          10.21    Male        Married     High School          

In [5]:
# Store important values
ntrain = train.shape[0]
ntest = test.shape[0]

print(f"Training samples: {ntrain}")
print(f"Test samples: {ntest}")

# Extract target
y_train = train['loan_paid_back'].values
print(f"\nTarget distribution:")
print(f"  Paid back (1): {(y_train == 1).sum()}")
print(f"  Defaulted (0): {(y_train == 0).sum()}")
print(f"  Ratio: {(y_train == 1).sum() / len(y_train) * 100:.1f}% paid back")

# Save test IDs
test_IDs = test['id'].copy()
print(f"\nTest IDs saved: {len(test_IDs)}")

# Combine train + test
all_data = pd.concat((train, test)).reset_index(drop=True)
print(f"Combined data shape: {all_data.shape}")
print("✓ Setup complete!")


Training samples: 593994
Test samples: 254569

Target distribution:
  Paid back (1): 474494
  Defaulted (0): 119500
  Ratio: 79.9% paid back

Test IDs saved: 254569
Combined data shape: (848563, 13)
✓ Setup complete!


In [6]:
all_data.drop(['id'], axis=1, inplace=True)

print(f"Shape after dropping ID: {all_data.shape}")
print(f"Remaining columns: {list(all_data.columns)}")
print("✓ ID dropped!")


Shape after dropping ID: (848563, 12)
Remaining columns: ['annual_income', 'debt_to_income_ratio', 'credit_score', 'loan_amount', 'interest_rate', 'gender', 'marital_status', 'education_level', 'employment_status', 'loan_purpose', 'grade_subgrade', 'loan_paid_back']
✓ ID dropped!


In [7]:
# Find all categorical columns
categorical_col = []
for col in all_data.columns:
    if all_data[col].dtype == 'object':  # Text columns
        categorical_col.append(col)
        print(f"\n'{col}' is categorical")
        print(f"  Unique values: {all_data[col].nunique()}")
        print(f"  Values: {list(all_data[col].unique()[:10])}")

print(f"\n\nTotal categorical columns: {len(categorical_col)}")
print(f"List: {categorical_col}")
print("✓ Categorical columns identified!")



'gender' is categorical
  Unique values: 3
  Values: ['Female', 'Male', 'Other']

'marital_status' is categorical
  Unique values: 4
  Values: ['Single', 'Married', 'Divorced', 'Widowed']

'education_level' is categorical
  Unique values: 5
  Values: ['High School', "Master's", "Bachelor's", 'PhD', 'Other']

'employment_status' is categorical
  Unique values: 5
  Values: ['Self-employed', 'Employed', 'Unemployed', 'Retired', 'Student']

'loan_purpose' is categorical
  Unique values: 8
  Values: ['Other', 'Debt consolidation', 'Home', 'Education', 'Vacation', 'Car', 'Medical', 'Business']

'grade_subgrade' is categorical
  Unique values: 30
  Values: ['C3', 'D3', 'C5', 'F1', 'D1', 'D5', 'C2', 'C1', 'F5', 'D4']


Total categorical columns: 6
List: ['gender', 'marital_status', 'education_level', 'employment_status', 'loan_purpose', 'grade_subgrade']
✓ Categorical columns identified!


In [8]:
print("---Encoding The variables---\n")
for col in categorical_col:
    print(f"Encoding'{col}'")
    unique_vals=all_data[col].unique()
    print(f"Before: {unique_vals}")
    
    all_data[col] = all_data[col].astype("category").cat.codes
    
    print(f"  After: {all_data[col].unique()}")
    print()

print("✓ All categorical variables encoded!")
print(f"\nNew data types:\n{all_data.dtypes}")
    


---Encoding The variables---

Encoding'gender'
Before: ['Female' 'Male' 'Other']
  After: [0 1 2]

Encoding'marital_status'
Before: ['Single' 'Married' 'Divorced' 'Widowed']
  After: [2 1 0 3]

Encoding'education_level'
Before: ['High School' "Master's" "Bachelor's" 'PhD' 'Other']
  After: [1 2 0 4 3]

Encoding'employment_status'
Before: ['Self-employed' 'Employed' 'Unemployed' 'Retired' 'Student']
  After: [2 0 4 1 3]

Encoding'loan_purpose'
Before: ['Other' 'Debt consolidation' 'Home' 'Education' 'Vacation' 'Car'
 'Medical' 'Business']
  After: [6 2 4 3 7 1 5 0]

Encoding'grade_subgrade'
Before: ['C3' 'D3' 'C5' 'F1' 'D1' 'D5' 'C2' 'C1' 'F5' 'D4' 'C4' 'D2' 'E5' 'B1'
 'B2' 'F4' 'A4' 'E1' 'F2' 'B4' 'E4' 'B3' 'E3' 'B5' 'E2' 'F3' 'A5' 'A3'
 'A1' 'A2']
  After: [12 17 14 25 15 19 11 10 29 18 13 16 24  5  6 28  3 20 26  8 23  7 22  9
 21 27  4  2  0  1]

✓ All categorical variables encoded!

New data types:
annual_income           float64
debt_to_income_ratio    float64
credit_score        

In [9]:
from sklearn.preprocessing import OneHotEncoder
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train['loan_paid_back'].values
test_IDs = test['id'].copy()

# Combine
all_data = pd.concat((train, test)).reset_index(drop=True)

# Drop ID
all_data.drop(['id'], axis=1, inplace=True)

# Find categorical columns
categorical_col = [col for col in all_data.columns if all_data[col].dtype == 'object']
print(f"Categorical columns: {categorical_col}")

# Encode
for col in categorical_col:
    all_data[col] = all_data[col].astype("category").cat.codes

# One-hot encode
from sklearn.preprocessing import OneHotEncoder
if len(categorical_col) > 0:
    ohe = OneHotEncoder(categories='auto', sparse_output=False)
    array_hot_encoded = ohe.fit_transform(all_data[categorical_col])
    data_hot_encoded = pd.DataFrame(array_hot_encoded, index=all_data.index)
    data_other_cols = all_data.drop(columns=categorical_col)
    all_data = pd.concat([data_hot_encoded, data_other_cols], axis=1)
    all_data.columns = all_data.columns.astype(str)

# Split back
train_data = all_data[:ntrain].copy()
test_data = all_data[ntrain:].copy()
train_data['loan_paid_back'] = y_train

print(f"\n Fixed!")
print(f"Train shape: {train_data.shape}")
print(f"Test shape: {test_data.shape}")


Categorical columns: ['gender', 'marital_status', 'education_level', 'employment_status', 'loan_purpose', 'grade_subgrade']

 Fixed!
Train shape: (593994, 61)
Test shape: (254569, 61)


In [10]:
print("Missing values:")
print(all_data.isnull().sum()[all_data.isnull().sum() > 0])

# Fill missing values
for col in all_data.columns:
    if all_data[col].isnull().sum() > 0:
        if all_data[col].dtype == 'object':
            all_data[col] = all_data[col].fillna(all_data[col].mode()[0])
        else:
            all_data[col] = all_data[col].fillna(all_data[col].median())

print("\nAfter filling:")
print(f"Total missing: {all_data.isnull().sum().sum()}")


Missing values:
loan_paid_back    254569
dtype: int64

After filling:
Total missing: 0


In [11]:
print("=" * 60)
print("VERIFICATION CHECK")
print("=" * 60)

print(f"\nTrain data:")
print(f"  Shape: {train_data.shape}")
print(f"  Columns: {list(train_data.columns)}")
print(f"  Has 'loan_paid_back': {'loan_paid_back' in train_data.columns}")

print(f"\nTest data:")
print(f"  Shape: {test_data.shape}")
print(f"  Columns: {list(test_data.columns)}")
print(f"  Has 'loan_paid_back': {'loan_paid_back' in test_data.columns}")

print(f"\nTarget variable (y_train):")
print(f"  Shape: {y_train.shape}")
print(f"  Unique values: {np.unique(y_train)}")
print(f"  Distribution:")
print(f"    0 (Defaulted): {(y_train == 0).sum()}")
print(f"    1 (Paid back): {(y_train == 1).sum()}")

print(f"\nTest IDs:")
print(f"  Shape: {test_IDs.shape}")
print(f"  First 5: {test_IDs.head().values}")

print("\n✓ Everything looks correct!")


VERIFICATION CHECK

Train data:
  Shape: (593994, 61)
  Columns: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', 'annual_income', 'debt_to_income_ratio', 'credit_score', 'loan_amount', 'interest_rate', 'loan_paid_back']
  Has 'loan_paid_back': True

Test data:
  Shape: (254569, 61)
  Columns: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', 'annual_income', 'debt_to_income_ratio', 'credit_score', 'loan_amount', 'interest_rate', 'loan_paid_back']
  Has

In [12]:
print("=" * 60)
print("FEATURE SELECTION")
print("=" * 60)

# Use original train_data with target
train_data['loan_paid_back'] = y_train

print(f"Train shape: {train_data.shape}")

# Calculate correlations
correlations = train_data.corr()['loan_paid_back'].abs().sort_values(ascending=False)

print("\nTop 15 correlated features:")
print(correlations.head(15))

# Select top 25 features
k = 25
top_features = correlations.head(k).index.tolist()

print(f"\nSelected {k} features:")
print(top_features)

# Filter data
train_data_selected = train_data[top_features].copy()
test_data_selected = test_data[[f for f in top_features if f != 'loan_paid_back']].copy()

print(f"\nAfter selection:")
print(f"  Train: {train_data_selected.shape}")
print(f"  Test: {test_data_selected.shape}")

print("\nFeature selection complete!")

FEATURE SELECTION
Train shape: (593994, 61)

Top 15 correlated features:
loan_paid_back          1.000000
16                      0.616839
12                      0.421611
debt_to_income_ratio    0.335680
credit_score            0.234560
15                      0.191178
interest_rate           0.131184
13                      0.083526
14                      0.077374
42                      0.065831
31                      0.055970
32                      0.054583
43                      0.052562
34                      0.052348
33                      0.051289
Name: loan_paid_back, dtype: float64

Selected 25 features:
['loan_paid_back', '16', '12', 'debt_to_income_ratio', 'credit_score', '15', 'interest_rate', '13', '14', '42', '31', '32', '43', '34', '33', '44', '41', '35', '30', '52', '48', '40', '47', '51', '50']

After selection:
  Train: (593994, 25)
  Test: (254569, 24)

Feature selection complete!


In [13]:
print("=" * 60)
print("PREPARE FEATURES AND TARGET")
print("=" * 60)

# Separate features and target
X_train = train_data_selected.drop('loan_paid_back', axis=1)
y_train_final = train_data_selected['loan_paid_back']

print(f"\nX_train shape: {X_train.shape}")
print(f"y_train shape: {y_train_final.shape}")
print(f"Test data shape: {test_data_selected.shape}")

# Verify columns match
print(f"\nX_train columns ({len(X_train.columns)}): {list(X_train.columns)}")
print(f"Test columns ({len(test_data_selected.columns)}): {list(test_data_selected.columns)}")

# Check if columns match
if len(X_train.columns) == len(test_data_selected.columns):
    if all(X_train.columns == test_data_selected.columns):
        print("✓ Columns match perfectly!")
    else:
        print("⚠ Column names don't match, aligning...")
        test_data_selected = test_data_selected[X_train.columns]
else:
    print(f"⚠ Different column counts!")

# Check target distribution
print("\nTarget distribution:")
print(f"  Paid back (1): {(y_train_final == 1).sum()} ({(y_train_final == 1).sum() / len(y_train_final) * 100:.1f}%)")
print(f"  Defaulted (0): {(y_train_final == 0).sum()} ({(y_train_final == 0).sum() / len(y_train_final) * 100:.1f}%)")

print("\n COMPLETE!")


PREPARE FEATURES AND TARGET

X_train shape: (593994, 24)
y_train shape: (593994,)
Test data shape: (254569, 24)

X_train columns (24): ['16', '12', 'debt_to_income_ratio', 'credit_score', '15', 'interest_rate', '13', '14', '42', '31', '32', '43', '34', '33', '44', '41', '35', '30', '52', '48', '40', '47', '51', '50']
Test columns (24): ['16', '12', 'debt_to_income_ratio', 'credit_score', '15', 'interest_rate', '13', '14', '42', '31', '32', '43', '34', '33', '44', '41', '35', '30', '52', '48', '40', '47', '51', '50']
✓ Columns match perfectly!

Target distribution:
  Paid back (1): 474494 (79.9%)
  Defaulted (0): 119500 (20.1%)

 COMPLETE!


In [14]:
from sklearn.preprocessing import StandardScaler

print("=" * 60)
print(" FEATURE SCALING")
print("=" * 60)

# Create scaler
print("\n1. Creating scaler...")
scaler = StandardScaler()

# Fit and transform train data
print("2. Fitting on train data and transforming...")
X_train_scaled = scaler.fit_transform(X_train)
print(f"   X_train_scaled shape: {X_train_scaled.shape}")

# Transform test data
print("3. Transforming test data...")
test_data_scaled = scaler.transform(test_data_selected)
print(f"   test_data_scaled shape: {test_data_scaled.shape}")

# Convert to DataFrame for easier handling
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
test_data_scaled = pd.DataFrame(test_data_scaled, columns=test_data_selected.columns)

print("\n4. Sample of scaled data (first 3 rows):")
print(X_train_scaled.head(3))

print(f"\n5. Scaling statistics:")
print(f"   X_train mean: {X_train_scaled.mean().mean():.6f} (should be ~0)")
print(f"   X_train std: {X_train_scaled.std().mean():.6f} (should be ~1)")

print("\nCOMPLETE!")


 FEATURE SCALING

1. Creating scaler...
2. Fitting on train data and transforming...
   X_train_scaled shape: (593994, 24)
3. Transforming test data...
   test_data_scaled shape: (254569, 24)

4. Sample of scaled data (first 3 rows):
         16        12  debt_to_income_ratio  credit_score       15  \
0 -0.342872 -1.773046             -0.535135      0.993849 -0.14317   
1 -0.342872  0.564001              0.660668     -0.810394 -0.14317   
2 -0.342872  0.564001             -0.345556      0.236067 -0.14317   

   interest_rate        13        14        42        31  ...        44  \
0       0.653899 -0.168784  3.212240 -0.256598 -0.161873  ... -0.239019   
1       0.280571 -0.168784 -0.311309  3.897149 -0.161873  ... -0.239019   
2      -1.292385 -0.168784 -0.311309 -0.256598 -0.161873  ... -0.239019   

        41        35        30        52        48        40        47  \
0 -0.24806 -0.314174 -0.157309 -0.092895 -0.117108 -0.257844 -0.109793   
1 -0.24806 -0.314174 -0.157309 -0.09

In [15]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
import time

print("=" * 70)
print("XGBOOST MODEL (FAST ALTERNATIVE)")
print("=" * 70)

print("\nTraining XGBoost with GPU P100...\n")

start_time = time.time()

xgb_model = XGBClassifier(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.02,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    gpu_id=0,
    tree_method='gpu_hist',  # GPU acceleration
    verbosity=0,
    use_label_encoder=False,
    eval_metric='logloss'
)

xgb_model.fit(X_train_scaled, y_train)

elapsed = time.time() - start_time

y_pred_xgb = xgb_model.predict_proba(X_train_scaled)[:, 1]
xgb_auc = roc_auc_score(y_train, y_pred_xgb)

print(f"\n{'='*70}")
print(f"XGBoost ROC-AUC: {xgb_auc:.4f}")
print(f"Training time: {elapsed:.2f} seconds")
print(f"{'='*70}\n")

# Predictions
xgb_predictions = xgb_model.predict_proba(test_data_scaled)[:, 1]

# Submission
xgb_submission = pd.DataFrame({
    'id': test_IDs.values,
    'loan_paid_back': xgb_predictions
})

xgb_submission.to_csv('submission_xgboost.csv', index=False)
print("✓ submission_xgboost.csv created!")


XGBOOST MODEL (FAST ALTERNATIVE)

Training XGBoost with GPU P100...


XGBoost ROC-AUC: 0.9203
Training time: 3.79 seconds

✓ submission_xgboost.csv created!


In [16]:
print("\n" + "=" * 70)
print("MAKING XGBOOST PREDICTIONS")
print("=" * 70)

xgb_predictions = xgb_model.predict_proba(test_data_scaled)[:, 1]

print(f"\n✓ Predictions: {len(xgb_predictions):,}")
print(f"✓ Min: {xgb_predictions.min():.6f}")
print(f"✓ Max: {xgb_predictions.max():.6f}")
print(f"✓ Mean: {xgb_predictions.mean():.6f}")



MAKING XGBOOST PREDICTIONS

✓ Predictions: 254,569
✓ Min: 0.001535
✓ Max: 0.999498
✓ Mean: 0.799660


In [17]:
print("\n" + "=" * 70)
print("CREATING XGBOOST SUBMISSION")
print("=" * 70)

xgb_submission = pd.DataFrame({
    'id': test_IDs.values,
    'loan_paid_back': xgb_predictions
})

xgb_submission.to_csv('submission_xgboost.csv', index=False)

print(f"\n✓ Submission file created!")
print(f"✓ Filename: submission_xgboost.csv")
print(f"✓ Shape: {xgb_submission.shape}")
print(f"\nFirst 10 rows:")
print(xgb_submission.head(10))

file_size = os.path.getsize('submission_xgboost.csv')
print(f"\n✓ File size: {file_size / 1024:.2f} KB")

print("\n" + "=" * 70)
print("✓ READY TO SUBMIT XGBOOST!")
print("=" * 70)



CREATING XGBOOST SUBMISSION

✓ Submission file created!
✓ Filename: submission_xgboost.csv
✓ Shape: (254569, 2)

First 10 rows:
       id  loan_paid_back
0  593994        0.906049
1  593995        0.984510
2  593996        0.473615
3  593997        0.927785
4  593998        0.952833
5  593999        0.969376
6  594000        0.982226
7  594001        0.964296
8  594002        0.941162
9  594003        0.003079

✓ File size: 4353.23 KB

✓ READY TO SUBMIT XGBOOST!
